This script walks through a forecast for SMR using the NN-operational model developed in the NASA-NW repo. 

# Import Modules

In [423]:
#high level modules
import os
import imp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [424]:
# custom modules
this_dir = "/Users/steeleb/Documents/GitHub/ats-data-driven-forecasting/NN-operational/arNN/"

imp.load_source("universals", os.path.join(this_dir, "universal_functions.py"))
from universals import load_pickle_file, calculate_vals


# Import models

In [425]:
model_dir = "/Users/steeleb/Documents/GitHub/ats-data-driven-forecasting/data/NN_train_val_test/SMR_forecast/models/leaky_basic_5/"

model_1 = load_pickle_file("model_1.pkl", model_dir)
model_2 = load_pickle_file("model_2.pkl", model_dir)
model_3 = load_pickle_file("model_3.pkl", model_dir)
model_4 = load_pickle_file("model_4.pkl", model_dir)
model_5 = load_pickle_file("model_5.pkl", model_dir)
model_6 = load_pickle_file("model_6.pkl", model_dir)
model_7 = load_pickle_file("model_7.pkl", model_dir)
model_8 = load_pickle_file("model_8.pkl", model_dir)


# Import data

In [426]:
data_dir = "/Users/steeleb/Documents/GitHub/ats-data-driven-forecasting/data/NN_train_val_test/SMR_forecast/"

test = pd.read_csv(os.path.join(data_dir, "t2022_standardized_v2024-10-28.csv"))
forecast = pd.read_csv(os.path.join(data_dir, "t2022_forecast_std_v2024-10-28.csv"))

test["date"] = pd.to_datetime(test["date"])
forecast["date"] = pd.to_datetime(forecast["date"])
forecast["forecast_date"] = pd.to_datetime(forecast["forecast_date"])

# we need the test columns to be the same as the forecast columns at the end of this, so grab the names for now
forecast_cols = test.columns

# and let's drop the observed temp data from the forecast columns, too
forecast_cols_less = forecast_cols.drop(["mean_1m_temp_degC", "mean_0_5m_temp_degC"])

# Create function to roll out forecast

In [427]:
""" def make_seven_day_forecast(date):
    
    print(date)
    date = pd.to_datetime(date)
    
    # get the forecast data from a specific date
    fore = forecast[forecast["forecast_date"] == date].copy()
    # earliest date is noon met for forecast. Calculate the difference in dates between the reported date and the forecast date and add as a column called "offset"
    fore["offset"] = (fore["date"] - fore["forecast_date"]).dt.days
    # remove the forecast date column
    fore = fore.drop(columns=["forecast_date"])
    
    # we'll run a forecast for each day, since the following day's forecast will be based on the previous day's forecast
    for d in range(0, 7):
        # Setup for the iteration
        print("Forecasting day: ", d+1)
        # set the forecast date
        forecast_date = pd.to_datetime(date) + pd.DateOffset(days=d)
        obs = test[test["date"] == forecast_date].copy()
        
        # the first day will be a bit different from subsequent days
        if d == 0:
            # remove the noon met data
            obs = obs.drop(columns=["noon_air_temp", "noon_ave_wind", "noon_solar_rad"])
            # grab the forecast data for the offset date
            fore_select = fore[fore["offset"] == d].copy()
            
            obs_fore = obs.join(fore_select.set_index("date"), on="date")
            obs_fore = obs_fore.drop(columns=["offset"])
            obs_fore = obs_fore.set_index("number")
            # now reorganize the columns to match the input columns
            obs_fore = obs_fore[forecast_cols]
            
            # preprocess the data into labels and features
            features = obs_fore.drop(columns = ["date", "mean_1m_temp_degC", "mean_0_5m_temp_degC"])
            
            # make the forecast for each perturbation
            pred_1 = model_1.predict(features)
            pred_2 = model_2.predict(features)
            pred_3 = model_3.predict(features)
            pred_4 = model_4.predict(features)
            pred_5 = model_5.predict(features)
            pred_6 = model_6.predict(features)
            pred_7 = model_7.predict(features)
            pred_8 = model_8.predict(features)

            # and now we need to create the dataframe for the next iteration
            # first, create a dataframe with the forecast data
            for i, pred in enumerate([pred_1, pred_2, pred_3, pred_4, pred_5, pred_6, pred_7, pred_8], start=1):
                forecasted_temp = pd.DataFrame(columns=['date', 'perturbation', 'model', 'mean_1m_temp_degC', 'mean_0_5m_temp_degC'])
                forecasted_temp["perturbation"] = obs_fore.index
                forecasted_temp['model'] = i
                forecasted_temp["mean_1m_temp_degC"] = [p[0] for p in pred]
                forecasted_temp["mean_0_5m_temp_degC"] = [p[1] for p in pred]
                forecasted_temp["date"] = forecast_date
                
                # Append to the main dataframe (or create it if it doesn't exist)
                if 'all_forecasts' in locals():
                    all_forecasts = pd.concat([all_forecasts, forecasted_temp])
                else:
                    all_forecasts = forecasted_temp.copy()
            
            
        
        elif d == 1:
            # remove the noon met data and the observed temperature data from yesterday and today (we'll replaced these with forecasted data)
            obs = obs.drop(columns=["noon_air_temp", "noon_ave_wind", "noon_solar_rad",
                                    "noon_air_temp_m1", "noon_ave_wind_m1", "noon_solar_rad_m1",
                                    "mean_1m_temp_degC", "mean_0_5m_temp_degC",
                                    "mean_1m_temp_degC_m1", "mean_0_5m_temp_degC_m1"])
            
            # grab the forecast data for the offset date the merge with the observed data
            fore_select = fore[fore["offset"] == d].copy()
            fore_select = fore_select.rename(columns={"number": "perturbation"})
            fore_select = fore_select.drop(columns=["offset"])
            
            # and the previous day
            m1_select = fore[fore["offset"] == d-1].copy()
            m1_select = m1_select.rename(columns={"number": "perturbation"})
            m1_select = m1_select.drop(columns=["offset"])
            m1_select["date"] = pd.to_datetime(m1_select["date"]) + pd.DateOffset(days=1)
            
            # join obs (one row) with fore_select (many rows) on the date column
            obs_fore = obs.join(fore_select.set_index("date"), on="date")
            obs_fore = obs_fore.reset_index()
            obs_fore = obs_fore.set_index(["date", "perturbation"])
            obs_fore = obs_fore.join(m1_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m1")
            
            # join the observational forecast data with the forecasted data from the previous day by date and perturbation
            to_forecast = all_forecasts.copy()
            to_forecast = to_forecast[to_forecast["date"] == forecast_date - pd.DateOffset(days=1)]
            to_forecast["date"] = pd.to_datetime(to_forecast["date"]) + pd.DateOffset(days=1)
            to_forecast.columns = to_forecast.columns.str.replace("degC", "degC_m1")
            to_forecast = to_forecast.set_index(["date", "perturbation"])
            to_forecast = to_forecast.join(obs_fore, on=["date", "perturbation"])

            # now we need to reorganize the columns to match the input columns, plus the model and peturbation colums
            # first, move the date and perturbation from the index to columns
            to_forecast = to_forecast.reset_index()
            # now change model and perturbation to the index
            to_forecast = to_forecast.set_index(["model", "perturbation"])
            # and now reorganize the columns to match the input columns
            to_forecast = to_forecast[forecast_cols_less]

            # and now we need to preprocess the data into features
            forecast_features = to_forecast.drop(columns = "date")

            # for each model, make the forecast and store the results
            # make a dataframe to store the forecasted data
            forecasted_temp = pd.DataFrame(columns=['date', 'perturbation', 'model', 'mean_1m_temp_degC', 'mean_0_5m_temp_degC'])
            
            # loop through the models
            for m in range(1, 9):
                # filter the forecast features for the model and drop the model column
                model_forecast_features = forecast_features[forecast_features.index.get_level_values(0) == m].copy()
                # make the forecast for each perturbation
                pred = eval(f"model_{m}").predict(model_forecast_features)
                forecasted_temp["perturbation"] = model_forecast_features.index.get_level_values(1)
                forecasted_temp['model'] = m
                forecasted_temp["mean_1m_temp_degC"] = [p[0] for p in pred]
                forecasted_temp["mean_0_5m_temp_degC"] = [p[1] for p in pred]
                forecasted_temp["date"] = forecast_date

                # Append to a dataframe (or create it if it doesn't exist)
                if 'forecasted_date' in locals():
                    forecasted_date = pd.concat([forecasted_date, forecasted_temp])
                else:
                    forecasted_date = forecasted_temp.copy()
            
            # Append to the main dataframe
            all_forecasts = pd.concat([all_forecasts, forecasted_date])
        
            # remove forecasted_date from memory
            del forecasted_date

        elif d == 2:
            # remove the noon met data and the observed temperature data from yesterday and today (we'll replaced these with forecasted data)
            obs = obs.drop(columns=["noon_air_temp", "noon_ave_wind", "noon_solar_rad",
                                    "noon_air_temp_m1", "noon_ave_wind_m1", "noon_solar_rad_m1",
                                    "noon_air_temp_m2", "noon_ave_wind_m2", "noon_solar_rad_m2",
                                    "mean_1m_temp_degC", "mean_0_5m_temp_degC",
                                    "mean_1m_temp_degC_m1", "mean_0_5m_temp_degC_m1", 
                                    "mean_1m_temp_degC_m2", "mean_0_5m_temp_degC_m2"])
            
            # grab the forecast data for the offset date the merge with the observed data
            fore_select = fore[fore["offset"] == d].copy()
            fore_select = fore_select.rename(columns={"number": "perturbation"})
            fore_select = fore_select.drop(columns=["offset"]) 
            
            # and the previous day
            m1_select = fore[fore["offset"] == d-1].copy()
            m1_select = m1_select.rename(columns={"number": "perturbation"})
            m1_select = m1_select.drop(columns=["offset"])
            m1_select["date"] = pd.to_datetime(m1_select["date"]) + pd.DateOffset(days=1)

            # and two days prior
            m2_select = fore[fore["offset"] == d-2].copy()
            m2_select = m2_select.rename(columns={"number": "perturbation"})
            m2_select = m2_select.drop(columns=["offset"])
            m2_select["date"] = pd.to_datetime(m2_select["date"]) + pd.DateOffset(days=2)

            # join obs (one row) with fore_select (many rows) on the date column
            obs_fore = obs.join(fore_select.set_index(["date"]), on=["date"])
            obs_fore = obs_fore.reset_index()
            obs_fore = obs_fore.set_index(["date", "perturbation"])
            obs_fore = obs_fore.join(m1_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m1")
            obs_fore = obs_fore.join(m2_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m2")
            
            # add yesterday's temp forecast to the observational forecast data by date and perturbation
            m1_forecast = all_forecasts.copy()
            m1_forecast = m1_forecast[m1_forecast["date"] == forecast_date - pd.DateOffset(days=1)]
            m1_forecast["date"] = pd.to_datetime(m1_forecast["date"]) + pd.DateOffset(days=1)
            m1_forecast.columns = m1_forecast.columns.str.replace("degC", "degC_m1")
            m1_forecast = m1_forecast.set_index(["date", "perturbation"])
            m1_forecast = m1_forecast.join(obs_fore, on=["date", "perturbation"])
            m1_forecast = m1_forecast.reset_index()
            m1_forecast = m1_forecast.set_index(["date", "perturbation", "model"])

            # and two days prior
            to_forecast = all_forecasts.copy()
            to_forecast = to_forecast[to_forecast["date"] == forecast_date - pd.DateOffset(days=2)]
            to_forecast["date"] = pd.to_datetime(to_forecast["date"]) + pd.DateOffset(days=2)
            to_forecast.columns = to_forecast.columns.str.replace("degC", "degC_m2")
            to_forecast = to_forecast.set_index(["date", "perturbation", "model"])
            to_forecast = to_forecast.join(m1_forecast, on=["date", "perturbation", "model"])

            # now we need to reorganize the columns to match the input columns, plus the model and peturbation colums
            # first, move the date and perturbation from the index to columns
            to_forecast = to_forecast.reset_index()
            # now change model and perturbation to the index
            to_forecast = to_forecast.set_index(["model", "perturbation"])
            # and now reorganize the columns to match the input columns
            to_forecast = to_forecast[forecast_cols_less]

            # and now we need to preprocess the data into features
            forecast_features = to_forecast.drop(columns = "date")
            # for each model, make the forecast and store the results
            # make a dataframe to store the forecasted data
            forecasted_temp = pd.DataFrame(columns=['date', 'perturbation', 'model', 'mean_1m_temp_degC', 'mean_0_5m_temp_degC'])
            
            # loop through the models
            for m in range(1, 9):
                # filter the forecast features for the model and drop the model column
                model_forecast_features = forecast_features[forecast_features.index.get_level_values(0) == m].copy()
                # make the forecast for each perturbation
                pred = eval(f"model_{m}").predict(model_forecast_features)
                forecasted_temp["perturbation"] = model_forecast_features.index.get_level_values(1)
                forecasted_temp['model'] = m
                forecasted_temp["mean_1m_temp_degC"] = [p[0] for p in pred]
                forecasted_temp["mean_0_5m_temp_degC"] = [p[1] for p in pred]
                forecasted_temp["date"] = forecast_date

                # Append to a dataframe (or create it if it doesn't exist)
                if 'forecasted_date' in locals():
                    forecasted_date = pd.concat([forecasted_date, forecasted_temp])
                else:
                    forecasted_date = forecasted_temp.copy()
            
            # Append to the main dataframe
            all_forecasts = pd.concat([all_forecasts, forecasted_date])

            # remove forecasted_date from memory
            del forecasted_date

        elif d == 3:
            
            # remove the noon met data and the observed temperature data from yesterday and today (we'll replaced these with forecasted data)
            obs = obs.drop(columns=["noon_air_temp", "noon_ave_wind", "noon_solar_rad",
                                    "noon_air_temp_m1", "noon_ave_wind_m1", "noon_solar_rad_m1",
                                    "noon_air_temp_m2", "noon_ave_wind_m2", "noon_solar_rad_m2",
                                    "noon_air_temp_m3", "noon_ave_wind_m3", "noon_solar_rad_m3",
                                    "mean_1m_temp_degC", "mean_0_5m_temp_degC",
                                    "mean_1m_temp_degC_m1", "mean_0_5m_temp_degC_m1", 
                                    "mean_1m_temp_degC_m2", "mean_0_5m_temp_degC_m2",
                                    "mean_1m_temp_degC_m3", "mean_0_5m_temp_degC_m3"])
            
            # grab the forecast data for the offset date the merge with the observed data
            fore_select = fore[fore["offset"] == d].copy()
            fore_select = fore_select.rename(columns={"number": "perturbation"})
            fore_select = fore_select.drop(columns=["offset"])
            
            # and the previous day
            m1_select = fore[fore["offset"] == d-1].copy()
            m1_select = m1_select.rename(columns={"number": "perturbation"})
            m1_select = m1_select.drop(columns=["offset"])
            m1_select["date"] = pd.to_datetime(m1_select["date"]) + pd.DateOffset(days=1)

            # and two days prior
            m2_select = fore[fore["offset"] == d-2].copy()
            m2_select = m2_select.rename(columns={"number": "perturbation"})
            m2_select = m2_select.drop(columns=["offset"])
            m2_select["date"] = pd.to_datetime(m2_select["date"]) + pd.DateOffset(days=2)

            # and three days prior
            m3_select = fore[fore["offset"] == d-3].copy()
            m3_select = m3_select.rename(columns={"number": "perturbation"})
            m3_select = m3_select.drop(columns=["offset"])
            m3_select["date"] = pd.to_datetime(m3_select["date"]) + pd.DateOffset(days=3)
            
            # join obs (one row) with fore_select (many rows) on the date column
            obs_fore = obs.join(fore_select.set_index("date"), on="date")
            obs_fore = obs_fore.reset_index()
            obs_fore = obs_fore.set_index(["date", "perturbation"])
            obs_fore = obs_fore.join(m1_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m1")
            obs_fore = obs_fore.join(m2_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m2")
            obs_fore = obs_fore.join(m3_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m3")
            
            # add yesterday's temp forecast to the observational forecast data by date and perturbation
            m1_forecast = all_forecasts.copy()
            m1_forecast = m1_forecast[m1_forecast["date"] == forecast_date - pd.DateOffset(days=1)]
            m1_forecast["date"] = pd.to_datetime(m1_forecast["date"]) + pd.DateOffset(days=1)
            m1_forecast.columns = m1_forecast.columns.str.replace("degC", "degC_m1")
            m1_forecast = m1_forecast.set_index(["date", "perturbation"])
            m1_forecast = m1_forecast.join(obs_fore, on=["date", "perturbation"])
            m1_forecast = m1_forecast.reset_index()
            m1_forecast = m1_forecast.set_index(["date", "perturbation", "model"])

            # and two days prior
            m2_forecast = all_forecasts.copy()
            m2_forecast = m2_forecast[m2_forecast["date"] == forecast_date - pd.DateOffset(days=2)]
            m2_forecast["date"] = pd.to_datetime(m2_forecast["date"]) + pd.DateOffset(days=2)
            m2_forecast.columns = m2_forecast.columns.str.replace("degC", "degC_m2")
            m2_forecast = m2_forecast.set_index(["date", "perturbation", "model"])
            m2_forecast = m2_forecast.join(m1_forecast, on=["date", "perturbation", "model"])

            # and three days prior
            to_forecast = all_forecasts.copy()
            to_forecast = to_forecast[to_forecast["date"] == forecast_date - pd.DateOffset(days=3)]
            to_forecast["date"] = pd.to_datetime(to_forecast["date"]) + pd.DateOffset(days=3)
            to_forecast.columns = to_forecast.columns.str.replace("degC", "degC_m3")
            to_forecast = to_forecast.set_index(["date", "perturbation", "model"])
            to_forecast = to_forecast.join(m2_forecast, on=["date", "perturbation", "model"])

            # now we need to reorganize the columns to match the input columns, plus the model and peturbation colums
            # first, move the date and perturbation from the index to columns
            to_forecast = to_forecast.reset_index()
            # now change model and perturbation to the index
            to_forecast = to_forecast.set_index(["model", "perturbation"])
            # and now reorganize the columns to match the input columns
            to_forecast = to_forecast[forecast_cols_less]

            # and now we need to preprocess the data into features
            forecast_features = to_forecast.drop(columns = "date")

            # for each model, make the forecast and store the results
            # make a dataframe to store the forecasted data
            forecasted_temp = pd.DataFrame(columns=['date', 'perturbation', 'model', 'mean_1m_temp_degC', 'mean_0_5m_temp_degC'])
            
            # loop through the models
            for m in range(1, 9):
                # filter the forecast features for the model and drop the model column
                model_forecast_features = forecast_features[forecast_features.index.get_level_values(0) == m].copy()
                # make the forecast for each perturbation
                pred = eval(f"model_{m}").predict(model_forecast_features)
                forecasted_temp["perturbation"] = model_forecast_features.index.get_level_values(1)
                forecasted_temp['model'] = m
                forecasted_temp["mean_1m_temp_degC"] = [p[0] for p in pred]
                forecasted_temp["mean_0_5m_temp_degC"] = [p[1] for p in pred]
                forecasted_temp["date"] = forecast_date

                # Append to a dataframe (or create it if it doesn't exist)
                if 'forecasted_date' in locals():
                    forecasted_date = pd.concat([forecasted_date, forecasted_temp])
                else:
                    forecasted_date = forecasted_temp.copy()
            
            # Append to the main dataframe
            all_forecasts = pd.concat([all_forecasts, forecasted_date])
            
            # remove forecasted_date from memory
            del forecasted_date

        elif d == 4:
       
            # remove the noon met data and the observed temperature data from yesterday and today (we'll replaced these with forecasted data)
            obs = obs.drop(columns=["noon_air_temp", "noon_ave_wind", "noon_solar_rad",
                                    "noon_air_temp_m1", "noon_ave_wind_m1", "noon_solar_rad_m1",
                                    "noon_air_temp_m2", "noon_ave_wind_m2", "noon_solar_rad_m2",
                                    "noon_air_temp_m3", "noon_ave_wind_m3", "noon_solar_rad_m3",
                                    "noon_air_temp_m4", "noon_ave_wind_m4", "noon_solar_rad_m4",
                                    "mean_1m_temp_degC", "mean_0_5m_temp_degC",
                                    "mean_1m_temp_degC_m1", "mean_0_5m_temp_degC_m1", 
                                    "mean_1m_temp_degC_m2", "mean_0_5m_temp_degC_m2",
                                    "mean_1m_temp_degC_m3", "mean_0_5m_temp_degC_m3"])
            
            # grab the forecast data for the offset date the merge with the observed data
            fore_select = fore[fore["offset"] == d].copy()
            fore_select = fore_select.rename(columns={"number": "perturbation"})
            fore_select = fore_select.drop(columns=["offset"])
            
            # and the previous day
            m1_select = fore[fore["offset"] == d-1].copy()
            m1_select = m1_select.rename(columns={"number": "perturbation"})
            m1_select = m1_select.drop(columns=["offset"])
            m1_select["date"] = pd.to_datetime(m1_select["date"]) + pd.DateOffset(days=1)

            # and two days prior
            m2_select = fore[fore["offset"] == d-2].copy()
            m2_select = m2_select.rename(columns={"number": "perturbation"})
            m2_select = m2_select.drop(columns=["offset"])
            m2_select["date"] = pd.to_datetime(m2_select["date"]) + pd.DateOffset(days=2)

            # and three days prior
            m3_select = fore[fore["offset"] == d-3].copy()
            m3_select = m3_select.rename(columns={"number": "perturbation"})
            m3_select = m3_select.drop(columns=["offset"])
            m3_select["date"] = pd.to_datetime(m3_select["date"]) + pd.DateOffset(days=3)

            # and four days prior
            m4_select = fore[fore["offset"] == d-4].copy()
            m4_select = m4_select.rename(columns={"number": "perturbation"})
            m4_select = m4_select.drop(columns=["offset"])
            m4_select["date"] = pd.to_datetime(m4_select["date"]) + pd.DateOffset(days=4)
            
            # join obs (one row) with fore_select (many rows) on the date column
            obs_fore = obs.join(fore_select.set_index("date"), on="date")
            obs_fore = obs_fore.reset_index()
            obs_fore = obs_fore.set_index(["date", "perturbation"])
            obs_fore = obs_fore.join(m1_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m1")
            obs_fore = obs_fore.join(m2_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m2")
            obs_fore = obs_fore.join(m3_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m3")
            obs_fore = obs_fore.join(m4_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m4")
            
            # add yesterday's temp forecast to the observational forecast data by date and perturbation
            m1_forecast = all_forecasts.copy()
            m1_forecast = m1_forecast[m1_forecast["date"] == forecast_date - pd.DateOffset(days=1)]
            m1_forecast["date"] = pd.to_datetime(m1_forecast["date"]) + pd.DateOffset(days=1)
            m1_forecast.columns = m1_forecast.columns.str.replace("degC", "degC_m1")
            m1_forecast = m1_forecast.set_index(["date", "perturbation"])
            m1_forecast = m1_forecast.join(obs_fore, on=["date", "perturbation"])
            m1_forecast = m1_forecast.reset_index()
            m1_forecast = m1_forecast.set_index(["date", "perturbation", "model"])

            # and two days prior
            m2_forecast = all_forecasts.copy()
            m2_forecast = m2_forecast[m2_forecast["date"] == forecast_date - pd.DateOffset(days=2)]
            m2_forecast["date"] = pd.to_datetime(m2_forecast["date"]) + pd.DateOffset(days=2)
            m2_forecast.columns = m2_forecast.columns.str.replace("degC", "degC_m2")
            m2_forecast = m2_forecast.set_index(["date", "perturbation", "model"])
            m2_forecast = m2_forecast.join(m1_forecast, on=["date", "perturbation", "model"])

            # and three days prior
            to_forecast = all_forecasts.copy()
            to_forecast = to_forecast[to_forecast["date"] == forecast_date - pd.DateOffset(days=3)]
            to_forecast["date"] = pd.to_datetime(to_forecast["date"]) + pd.DateOffset(days=3)
            to_forecast.columns = to_forecast.columns.str.replace("degC", "degC_m3")
            to_forecast = to_forecast.set_index(["date", "perturbation", "model"])
            to_forecast = to_forecast.join(m2_forecast, on=["date", "perturbation", "model"])

            # now we need to reorganize the columns to match the input columns, plus the model and peturbation colums
            # first, move the date and perturbation from the index to columns
            to_forecast = to_forecast.reset_index()
            # now change model and perturbation to the index
            to_forecast = to_forecast.set_index(["model", "perturbation"])
            # and now reorganize the columns to match the input columns
            to_forecast = to_forecast[forecast_cols_less]

            # and now we need to preprocess the data into features
            forecast_features = to_forecast.drop(columns = "date")

            # for each model, make the forecast and store the results
            # make a dataframe to store the forecasted data
            forecasted_temp = pd.DataFrame(columns=['date', 'perturbation', 'model', 'mean_1m_temp_degC', 'mean_0_5m_temp_degC'])
            
            # loop through the models
            for m in range(1, 9):
                # filter the forecast features for the model and drop the model column
                model_forecast_features = forecast_features[forecast_features.index.get_level_values(0) == m].copy()
                # make the forecast for each perturbation
                pred = eval(f"model_{m}").predict(model_forecast_features)
                forecasted_temp["perturbation"] = model_forecast_features.index.get_level_values(1)
                forecasted_temp['model'] = m
                forecasted_temp["mean_1m_temp_degC"] = [p[0] for p in pred]
                forecasted_temp["mean_0_5m_temp_degC"] = [p[1] for p in pred]
                forecasted_temp["date"] = forecast_date

                # Append to a dataframe (or create it if it doesn't exist)
                if 'forecasted_date' in locals():
                    forecasted_date = pd.concat([forecasted_date, forecasted_temp])
                else:
                    forecasted_date = forecasted_temp.copy()
            
            # Append to the main dataframe
            all_forecasts = pd.concat([all_forecasts, forecasted_date])
            
            # remove forecasted_date from memory
            del forecasted_date

        elif d == 5:
           
            # remove the noon met data and the observed temperature data from yesterday and today (we'll replaced these with forecasted data)
            obs = obs.drop(columns=["noon_air_temp", "noon_ave_wind", "noon_solar_rad",
                                    "noon_air_temp_m1", "noon_ave_wind_m1", "noon_solar_rad_m1",
                                    "noon_air_temp_m2", "noon_ave_wind_m2", "noon_solar_rad_m2",
                                    "noon_air_temp_m3", "noon_ave_wind_m3", "noon_solar_rad_m3",
                                    "noon_air_temp_m4", "noon_ave_wind_m4", "noon_solar_rad_m4",
                                    "noon_air_temp_m5", "noon_ave_wind_m5", "noon_solar_rad_m5",
                                    "mean_1m_temp_degC", "mean_0_5m_temp_degC",
                                    "mean_1m_temp_degC_m1", "mean_0_5m_temp_degC_m1", 
                                    "mean_1m_temp_degC_m2", "mean_0_5m_temp_degC_m2",
                                    "mean_1m_temp_degC_m3", "mean_0_5m_temp_degC_m3"])
            
            # grab the forecast data for the offset date the merge with the observed data
            fore_select = fore[fore["offset"] == d].copy()
            fore_select = fore_select.rename(columns={"number": "perturbation"})
            fore_select = fore_select.drop(columns=["offset"])
            
            # and the previous day
            m1_select = fore[fore["offset"] == d-1].copy()
            m1_select = m1_select.rename(columns={"number": "perturbation"})
            m1_select = m1_select.drop(columns=["offset"])
            m1_select["date"] = pd.to_datetime(m1_select["date"]) + pd.DateOffset(days=1)

            # and two days prior
            m2_select = fore[fore["offset"] == d-2].copy()
            m2_select = m2_select.rename(columns={"number": "perturbation"})
            m2_select = m2_select.drop(columns=["offset"])
            m2_select["date"] = pd.to_datetime(m2_select["date"]) + pd.DateOffset(days=2)

            # and three days prior
            m3_select = fore[fore["offset"] == d-3].copy()
            m3_select = m3_select.rename(columns={"number": "perturbation"})
            m3_select = m3_select.drop(columns=["offset"])
            m3_select["date"] = pd.to_datetime(m3_select["date"]) + pd.DateOffset(days=3)

            # and four days prior
            m4_select = fore[fore["offset"] == d-4].copy()
            m4_select = m4_select.rename(columns={"number": "perturbation"})
            m4_select = m4_select.drop(columns=["offset"])
            m4_select["date"] = pd.to_datetime(m4_select["date"]) + pd.DateOffset(days=4)

            # and five days prior
            m5_select = fore[fore["offset"] == d-5].copy()
            m5_select = m5_select.rename(columns={"number": "perturbation"})
            m5_select = m5_select.drop(columns=["offset"])
            m5_select["date"] = pd.to_datetime(m5_select["date"]) + pd.DateOffset(days=5)
            
            # join obs (one row) with fore_select (many rows) on the date column
            obs_fore = obs.join(fore_select.set_index("date"), on=["date"])
            obs_fore = obs_fore.reset_index()
            obs_fore = obs_fore.set_index(["date", "perturbation"])
            obs_fore = obs_fore.join(m1_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m1")
            obs_fore = obs_fore.join(m2_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m2")
            obs_fore = obs_fore.join(m3_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m3")
            obs_fore = obs_fore.join(m4_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m4")
            obs_fore = obs_fore.join(m5_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m5")
            
            # add yesterday's temp forecast to the observational forecast data by date and perturbation
            m1_forecast = all_forecasts.copy()
            m1_forecast = m1_forecast[m1_forecast["date"] == forecast_date - pd.DateOffset(days=1)]
            m1_forecast["date"] = pd.to_datetime(m1_forecast["date"]) + pd.DateOffset(days=1)
            m1_forecast.columns = m1_forecast.columns.str.replace("degC", "degC_m1")
            m1_forecast = m1_forecast.set_index(["date", "perturbation"])
            m1_forecast = m1_forecast.join(obs_fore, on=["date", "perturbation"])
            m1_forecast = m1_forecast.reset_index()
            m1_forecast = m1_forecast.set_index(["date", "perturbation", "model"])

            # and two days prior
            m2_forecast = all_forecasts.copy()
            m2_forecast = m2_forecast[m2_forecast["date"] == forecast_date - pd.DateOffset(days=2)]
            m2_forecast["date"] = pd.to_datetime(m2_forecast["date"]) + pd.DateOffset(days=2)
            m2_forecast.columns = m2_forecast.columns.str.replace("degC", "degC_m2")
            m2_forecast = m2_forecast.set_index(["date", "perturbation", "model"])
            m2_forecast = m2_forecast.join(m1_forecast, on=["date", "perturbation", "model"])

            # and three days prior
            to_forecast = all_forecasts.copy()
            to_forecast = to_forecast[to_forecast["date"] == forecast_date - pd.DateOffset(days=3)]
            to_forecast["date"] = pd.to_datetime(to_forecast["date"]) + pd.DateOffset(days=3)
            to_forecast.columns = to_forecast.columns.str.replace("degC", "degC_m3")
            to_forecast = to_forecast.set_index(["date", "perturbation", "model"])
            to_forecast = to_forecast.join(m2_forecast, on=["date", "perturbation", "model"])

            # now we need to reorganize the columns to match the input columns, plus the model and peturbation colums
            # first, move the date and perturbation from the index to columns
            to_forecast = to_forecast.reset_index()
            # now change model and perturbation to the index
            to_forecast = to_forecast.set_index(["model", "perturbation"])
            # and now reorganize the columns to match the input columns
            to_forecast = to_forecast[forecast_cols_less]

            # and now we need to preprocess the data into features
            forecast_features = to_forecast.drop(columns = "date")

            # for each model, make the forecast and store the results
            # make a dataframe to store the forecasted data
            forecasted_temp = pd.DataFrame(columns=['date', 'perturbation', 'model', 'mean_1m_temp_degC', 'mean_0_5m_temp_degC'])
            
            # loop through the models
            for m in range(1, 9):
                # filter the forecast features for the model and drop the model column
                model_forecast_features = forecast_features[forecast_features.index.get_level_values(0) == m].copy()
                # make the forecast for each perturbation
                pred = eval(f"model_{m}").predict(model_forecast_features)
                forecasted_temp["perturbation"] = model_forecast_features.index.get_level_values(1)
                forecasted_temp['model'] = m
                forecasted_temp["mean_1m_temp_degC"] = [p[0] for p in pred]
                forecasted_temp["mean_0_5m_temp_degC"] = [p[1] for p in pred]
                forecasted_temp["date"] = forecast_date

                # Append to a dataframe (or create it if it doesn't exist)
                if 'forecasted_date' in locals():
                    forecasted_date = pd.concat([forecasted_date, forecasted_temp])
                else:
                    forecasted_date = forecasted_temp.copy()
            
            # Append to the main dataframe
            all_forecasts = pd.concat([all_forecasts, forecasted_date])
            
            # remove forecasted_date from memory
            del forecasted_date

        elif d == 6:

            # remove the noon met data and the observed temperature data from yesterday and today (we'll replaced these with forecasted data)
            obs = obs.drop(columns=["noon_air_temp", "noon_ave_wind", "noon_solar_rad",
                                    "noon_air_temp_m1", "noon_ave_wind_m1", "noon_solar_rad_m1",
                                    "noon_air_temp_m2", "noon_ave_wind_m2", "noon_solar_rad_m2",
                                    "noon_air_temp_m3", "noon_ave_wind_m3", "noon_solar_rad_m3",
                                    "noon_air_temp_m4", "noon_ave_wind_m4", "noon_solar_rad_m4",
                                    "noon_air_temp_m5", "noon_ave_wind_m5", "noon_solar_rad_m5",
                                    "noon_air_temp_m6", "noon_ave_wind_m6", "noon_solar_rad_m6",
                                    "mean_1m_temp_degC", "mean_0_5m_temp_degC",
                                    "mean_1m_temp_degC_m1", "mean_0_5m_temp_degC_m1", 
                                    "mean_1m_temp_degC_m2", "mean_0_5m_temp_degC_m2",
                                    "mean_1m_temp_degC_m3", "mean_0_5m_temp_degC_m3"])
            
            # grab the forecast data for the offset date the merge with the observed data
            fore_select = fore[fore["offset"] == d].copy()
            fore_select = fore_select.rename(columns={"number": "perturbation"})
            fore_select = fore_select.drop(columns=["offset"])
            
            # and the previous day
            m1_select = fore[fore["offset"] == d-1].copy()
            m1_select = m1_select.rename(columns={"number": "perturbation"})
            m1_select = m1_select.drop(columns=["offset"])
            m1_select["date"] = pd.to_datetime(m1_select["date"]) + pd.DateOffset(days=1)

            # and two days prior
            m2_select = fore[fore["offset"] == d-2].copy()
            m2_select = m2_select.rename(columns={"number": "perturbation"})
            m2_select = m2_select.drop(columns=["offset"])
            m2_select["date"] = pd.to_datetime(m2_select["date"]) + pd.DateOffset(days=2)

            # and three days prior
            m3_select = fore[fore["offset"] == d-3].copy()
            m3_select = m3_select.rename(columns={"number": "perturbation"})
            m3_select = m3_select.drop(columns=["offset"])
            m3_select["date"] = pd.to_datetime(m3_select["date"]) + pd.DateOffset(days=3)

            # and four days prior
            m4_select = fore[fore["offset"] == d-4].copy()
            m4_select = m4_select.rename(columns={"number": "perturbation"})
            m4_select = m4_select.drop(columns=["offset"])
            m4_select["date"] = pd.to_datetime(m4_select["date"]) + pd.DateOffset(days=4)

            # and five days prior
            m5_select = fore[fore["offset"] == d-5].copy()
            m5_select = m5_select.rename(columns={"number": "perturbation"})
            m5_select = m5_select.drop(columns=["offset"])
            m5_select["date"] = pd.to_datetime(m5_select["date"]) + pd.DateOffset(days=5)

            # and six days prior
            m6_select = fore[fore["offset"] == d-6].copy()
            m6_select = m6_select.rename(columns={"number": "perturbation"})
            m6_select = m6_select.drop(columns=["offset"])
            m6_select["date"] = pd.to_datetime(m6_select["date"]) + pd.DateOffset(days=6)
            
            # join obs (one row) with fore_select (many rows) on the date column
            obs_fore = obs.join(fore_select.set_index(["date"]), on=["date"])
            obs_fore = obs_fore.reset_index()
            obs_fore = obs_fore.set_index(["date", "perturbation"])
            obs_fore = obs_fore.join(m1_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m1")
            obs_fore = obs_fore.join(m2_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m2")
            obs_fore = obs_fore.join(m3_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m3")
            obs_fore = obs_fore.join(m4_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m4")
            obs_fore = obs_fore.join(m5_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m5")
            obs_fore = obs_fore.join(m6_select.set_index(["date", "perturbation"]), on=["date", "perturbation"], rsuffix="_m6")
            
            # add yesterday's temp forecast to the observational forecast data by date and perturbation
            m1_forecast = all_forecasts.copy()
            m1_forecast = m1_forecast[m1_forecast["date"] == forecast_date - pd.DateOffset(days=1)]
            m1_forecast["date"] = pd.to_datetime(m1_forecast["date"]) + pd.DateOffset(days=1)
            m1_forecast.columns = m1_forecast.columns.str.replace("degC", "degC_m1")
            m1_forecast = m1_forecast.set_index(["date", "perturbation"])
            m1_forecast = m1_forecast.join(obs_fore, on=["date", "perturbation"])
            m1_forecast = m1_forecast.reset_index()
            m1_forecast = m1_forecast.set_index(["date", "perturbation", "model"])

            # and two days prior
            m2_forecast = all_forecasts.copy()
            m2_forecast = m2_forecast[m2_forecast["date"] == forecast_date - pd.DateOffset(days=2)]
            m2_forecast["date"] = pd.to_datetime(m2_forecast["date"]) + pd.DateOffset(days=2)
            m2_forecast.columns = m2_forecast.columns.str.replace("degC", "degC_m2")
            m2_forecast = m2_forecast.set_index(["date", "perturbation", "model"])
            m2_forecast = m2_forecast.join(m1_forecast, on=["date", "perturbation", "model"])

            # and three days prior
            to_forecast = all_forecasts.copy()
            to_forecast = to_forecast[to_forecast["date"] == forecast_date - pd.DateOffset(days=3)]
            to_forecast["date"] = pd.to_datetime(to_forecast["date"]) + pd.DateOffset(days=3)
            to_forecast.columns = to_forecast.columns.str.replace("degC", "degC_m3")
            to_forecast = to_forecast.set_index(["date", "perturbation", "model"])
            to_forecast = to_forecast.join(m2_forecast, on=["date", "perturbation", "model"])

            # now we need to reorganize the columns to match the input columns, plus the model and peturbation colums
            # first, move the date and perturbation from the index to columns
            to_forecast = to_forecast.reset_index()
            # now change model and perturbation to the index
            to_forecast = to_forecast.set_index(["model", "perturbation"])
            # and now reorganize the columns to match the input columns
            to_forecast = to_forecast[forecast_cols_less]
            # and now we need to preprocess the data into features
            forecast_features = to_forecast.drop(columns = "date")
            
            # for each model, make the forecast and store the results
            # make a dataframe to store the forecasted data
            forecasted_temp = pd.DataFrame(columns=['date', 'perturbation', 'model', 'mean_1m_temp_degC', 'mean_0_5m_temp_degC'])
            
            # loop through the models
            for m in range(1, 9):
                # filter the forecast features for the model and drop the model column
                model_forecast_features = forecast_features[forecast_features.index.get_level_values(0) == m].copy()
                # make the forecast for each perturbation
                pred = eval(f"model_{m}").predict(model_forecast_features)
                forecasted_temp["perturbation"] = model_forecast_features.index.get_level_values(1)
                forecasted_temp['model'] = m
                forecasted_temp["mean_1m_temp_degC"] = [p[0] for p in pred]
                forecasted_temp["mean_0_5m_temp_degC"] = [p[1] for p in pred]
                forecasted_temp["date"] = forecast_date

                # Append to a dataframe (or create it if it doesn't exist)
                if 'forecasted_date' in locals():
                    forecasted_date = pd.concat([forecasted_date, forecasted_temp])
                else:
                    forecasted_date = forecasted_temp.copy()
            
            # Append to the main dataframe
            all_forecasts = pd.concat([all_forecasts, forecasted_date])

            # remove forecasted_date from memory
            del forecasted_date
        
    return all_forecasts
 """

' def make_seven_day_forecast(date):\n    \n    print(date)\n    date = pd.to_datetime(date)\n    \n    # get the forecast data from a specific date\n    fore = forecast[forecast["forecast_date"] == date].copy()\n    # earliest date is noon met for forecast. Calculate the difference in dates between the reported date and the forecast date and add as a column called "offset"\n    fore["offset"] = (fore["date"] - fore["forecast_date"]).dt.days\n    # remove the forecast date column\n    fore = fore.drop(columns=["forecast_date"])\n    \n    # we\'ll run a forecast for each day, since the following day\'s forecast will be based on the previous day\'s forecast\n    for d in range(0, 7):\n        # Setup for the iteration\n        print("Forecasting day: ", d+1)\n        # set the forecast date\n        forecast_date = pd.to_datetime(date) + pd.DateOffset(days=d)\n        obs = test[test["date"] == forecast_date].copy()\n        \n        # the first day will be a bit different from subsequ

### Data Preparation Functions

Here we just prepare the forecasted met data

In [428]:
def prepare_initial_forecast_data(date):
    # get the met forecast data from a specific date
    fore = forecast[forecast["forecast_date"] == date].copy()
    # caalculate the difference in dates between the date we are making a water temp forecast for
    # and the `forecast_date` of the met data and add as a column called "offset"
    fore["offset"] = (fore["date"] - fore["forecast_date"]).dt.days
    # drop the forecast_date column (which originated from the met forecast dataframe)
    fore = fore.drop(columns=["forecast_date"])
    return fore

Rality check: this should return a data frame with offset calculated for each forecasted day.

In [429]:
date = "2022-07-01"
fore = prepare_initial_forecast_data(date)
fore.head()

,date,number,noon_air_temp,noon_ave_wind,noon_solar_rad,offset
13237,2022-07-01,0,1.204289,-1.949879,0.350326,0
13238,2022-07-02,0,2.744055,-2.396813,1.058133,1
13239,2022-07-03,0,1.807250,-1.529991,0.454718,2
13240,2022-07-04,0,2.064673,-1.473007,0.561042,3
13241,2022-07-05,0,4.694532,-8.678423,1.068677,4


### Day-Specific Processing Functions

The function for the first day of the forecast is different from the rest of the days. This is because the first day of the forecast is the only day where we have the actual SMR values, and use them as initial conditions. 

In [430]:
def process_day_zero(obs, fore):
    # drop observed met data
    obs_less = obs.drop(columns=["noon_air_temp", "noon_ave_wind", "noon_solar_rad"])
    # assumes offset is 0 (aka d = 0)
    fore_select = fore[fore["offset"] == 0].copy()
    # join fore_select with obs, drop offset columns, and use the number column as index
    obs_fore = obs_less.join(fore_select.set_index("date"), on="date")
    obs_fore = obs_fore.drop(columns=["offset"]).set_index("number")
    return obs_fore[forecast_cols] # here we make sure the columns are in the correct order for the model


Reality check for first day preprocessing: date should be the same as `date` all perturbations (`number`) should be the same in the output `obs_fore`.

In [431]:
forecast_date = pd.to_datetime(date) + pd.DateOffset(days=0)
obs = test[test["date"] == forecast_date].copy()
obs_fore = process_day_zero(obs, fore)
forecast_date, obs_fore.head()


(Timestamp('2022-07-01 00:00:00'),
              date  mean_1m_temp_degC  mean_0_5m_temp_degC  \
 number                                                      
 0      2022-07-01           0.212988             0.360724   
 1      2022-07-01           0.212988             0.360724   
 2      2022-07-01           0.212988             0.360724   
 3      2022-07-01           0.212988             0.360724   
 4      2022-07-01           0.212988             0.360724   
 
         mean_1m_temp_degC_m1  mean_0_5m_temp_degC_m1  mean_1m_temp_degC_m2  \
 number                                                                       
 0                   0.122124                0.443163              0.130411   
 1                   0.122124                0.443163              0.130411   
 2                   0.122124                0.443163              0.130411   
 3                   0.122124                0.443163              0.130411   
 4                   0.122124                0.443163  

All subsequent days are more complicated. The model is run for each day, and the output is used as input for the next day, but the output from the day zero implementation has 31 x 50 some odd members.

Here is what needs to happen for each day:
   - use the forecast met data for d-offset for noon met data
   - drop the _m7 met data columns
   - rename the _m1, ..., _m6 columns to _m2, ..., _m7 for the met data columns
   - drop the _m3 column of the mean_1m_temp_degC and mean_0_5m_temp_degC
   - rename the _m1, _m2, _m3 columns to _m2, _m3
   - use the forecasted data from the previous day for mean_1m_temp_degC_m1 and mean_0_5m_temp_degC_m1
   - use the test data for all other columns (flow, chipmunk, north_fork)
  
This function gets iteratively larger as we add more days to the forecast, because we assimilate the forecasted data from the previous days as 'observed' or initial conditions for the following days.

In [450]:
# drop the columns that will be replaced with forecast data - this is dependent on the number of days to forecast
def drop_columns(obs, d):
    # we will always drop the observed met at noon and the observed temperature data
    cols = ["noon_air_temp", "noon_ave_wind", "noon_solar_rad", "mean_1m_temp_degC", "mean_0_5m_temp_degC"]
    for i in range(1, min(d + 1, 7)):
        # and then also, we will drop the met and temp data from the assimilated data where i is the number of days prior (m{i})
        cols.extend([f"noon_air_temp_m{i}", f"noon_ave_wind_m{i}", f"noon_solar_rad_m{i}"])
    for i in range(1, min(d + 1, 4)):
        # and then also, we will drop the met and temp data from the assimilated data where i is the number of days prior (m{i})
        cols.extend([f"mean_1m_temp_degC_m{i}", f"mean_0_5m_temp_degC_m{i}"])
    return obs.drop(columns=cols)

def met_data_selection(fore, d):
    # initialize an empty dictionary to store the selections
    selections = {}
    
    # Current day selection
    key = 'today'
    selections[key] = {}
    selections[key] = fore[fore["offset"] == d].copy()
    selections[key] = selections[key].rename(columns={"number": "perturbation"})
    selections[key] = selections[key].drop(columns=["offset"])
    
    for i in range(1, min(d + 1, 7)): # here, we want to stop after 6 days or the number of days we are forecasting at the most, whichever is less
        # define the key for the dictionary
        key = f'm{i}_select'
        selections[key] = {}
        selections[key] = fore[fore["offset"] == d - i].copy()
        selections[key] = selections[key].rename(columns={"number": "perturbation"})
        selections[key] = selections[key].drop(columns=["offset"])
        selections[key]["date"] = pd.to_datetime(selections[key]["date"]) + pd.DateOffset(days=i)
    
    return selections

def make_dataframe_from_selections(selections):
    # get the keys from the dictionary
    keys = list(selections.keys())
    # now we need to grab each key, and rename the columns adding '_mx' to the end of the column name
    for i in range(len(keys)):
        # the mx is stored in the key, so we can grab the number from the key
        mx = keys[i].split("_")[0]
        # as long as the key is not 'today', grab the column names that start with 'noon' and rename them
        if keys[i] != 'today':
            cols = selections[keys[i]].filter(regex='^noon').columns
            # and rename the columns with the suffix '_mx'
            selections[keys[i]] = selections[keys[i]].rename(columns={c: f"{c}_{mx}" for c in cols})
    # and now we need to concatenate the dataframes by the columns date and perturbation
    reindexed_selections = {key: df.set_index(['date', 'perturbation']) for key, df in selections.items()}
    # retrun the concatenated dataframes and reset the index
    return pd.concat(reindexed_selections.values(), axis=1).reset_index()
     

def make_forecast_ready(obs_less, selections, all_forecasts, forecast_date, d):
    
    # add previous days' temp forecast to the observational/forecast data by date and perturbation
    for i in range(1, min(d + 1, 4)):
        if i == 1:
            # Join observed data with selected met data
            obs_fore = obs_less.join(selections.set_index("date"), on="date")
            obs_fore = obs_fore.set_index(["date", "perturbation"])
            # make a copy of the forecasted data    
            prev_forecast = all_forecasts.copy()
            # grab the forecast data desired
            prev_forecast = prev_forecast[prev_forecast["date"] == forecast_date - pd.DateOffset(days=i)]
            # update the date column
            prev_forecast["date"] = pd.to_datetime(prev_forecast["date"]) + pd.DateOffset(days=i)
            # update the column names
            prev_forecast.columns = prev_forecast.columns.str.replace("degC", f"degC_m{i}")
            # set the index to join the forecast with the many rows of obs_fore
            prev_forecast = prev_forecast.set_index(["date", "perturbation"])
            prev_forecast = prev_forecast.join(obs_fore, on=["date", "perturbation"])
            # reset to join with the other previously-forecasted forecasted data
            prev_forecast = prev_forecast.reset_index()
            print(prev_forecast)
            prev_forecast = prev_forecast.set_index(["date", "perturbation", "model"])
            prev_forecasted = prev_forecast.copy()
            
        else:
            obs_fore = obs_less.join(selections.set_index(["date"]), on=["date"])
            obs_fore = obs_fore.set_index(["date", "perturbation"])
            prev_forecast = all_forecasts.copy()
            prev_forecast = prev_forecast[prev_forecast["date"] == forecast_date - pd.DateOffset(days=i)]
            prev_forecast["date"] = pd.to_datetime(prev_forecast["date"]) + pd.DateOffset(days=i)
            # update the column names to the correct offset
            prev_forecast.columns = prev_forecast.columns.str.replace("degC", f"degC_m{i}")
            print(prev_forecast)
            prev_forecast = prev_forecast.set_index(["date", "perturbation", "model"])
            # add to the previous forecasted data
            prev_forecasted = prev_forecasted.join(prev_forecast, on = ["date", "perturbation", "model"])

    # now we need to reorganize the columns to match the input columns, plus the model and peturbation colums
    # first, move the date and perturbation from the index to columns
    forecast_ready = prev_forecasted.reset_index().copy()
    # now change model and perturbation to the index, we want to preserve that information
    forecast_ready = forecast_ready.set_index(["model", "perturbation"])
    # filter for only the forecast date
    forecast_ready = forecast_ready[forecast_ready["date"] == forecast_date]
    # and now reorganize the columns to match the input columns
    forecast_ready = forecast_ready[forecast_cols_less].drop(columns = "date")
    
    return forecast_ready


Reality check: do these function outputs make sense?

In [433]:
# here, we shoulding have any m1 or m2 met or temp data
d = 2
obs_less = drop_columns(obs, d)
obs_less.columns

Index(['date', 'mean_1m_temp_degC_m3', 'mean_0_5m_temp_degC_m3', 'pump_q_m1',
       'pump_q_m2', 'sum_pump_q_p2', 'max_pump_q_p2', 'sum_pump_q_p7',
       'max_pump_q_p7', 'ave_chip_q_m1', 'max_chip_q_m1', 'ave_chip_q_m2',
       'max_chip_q_m2', 'mean_chip_q_p7', 'max_chip_q_p7', 'noon_air_temp_m3',
       'noon_air_temp_m4', 'noon_air_temp_m5', 'noon_air_temp_m6',
       'noon_air_temp_m7', 'noon_ave_wind_m3', 'noon_ave_wind_m4',
       'noon_ave_wind_m5', 'noon_ave_wind_m6', 'noon_ave_wind_m7',
       'noon_solar_rad_m3', 'noon_solar_rad_m4', 'noon_solar_rad_m5',
       'noon_solar_rad_m6', 'noon_solar_rad_m7', 'min_chip_q_m1',
       'min_chip_q_m2', 'min_chip_q_p7', 'NF_q_m1', 'NF_q_m2', 'mean_NF_q_p7',
       'max_NF_q_p7'],
      dtype='object')

In [434]:
# and we'll check to make sure the selected columns are correct for adding back in met data - since d = 2, we should have both m1 and m2 met data

selections = met_data_selection(fore, d)

selections

{'today':             date  perturbation  noon_air_temp  noon_ave_wind  noon_solar_rad
 13239 2022-07-03             0       1.807250      -1.529991        0.454718
 13246 2022-07-03             1       1.192633      -1.249993        0.048738
 13253 2022-07-03             2       2.697526      -0.612498        0.963957
 13260 2022-07-03             3       1.925265      -3.510717        0.004560
 13267 2022-07-03             4       0.104025      -1.528424       -0.700262
 13274 2022-07-03             5       1.794290       0.013174        0.779485
 13281 2022-07-03             6       0.815304      -2.470261        0.464304
 13288 2022-07-03             7       2.361078      -1.615342        1.047605
 13295 2022-07-03             8       2.414022      -0.942392        1.089813
 13302 2022-07-03             9       4.487174      -5.731785        1.111012
 13309 2022-07-03            10       1.877148      -2.297216       -0.013000
 13316 2022-07-03            11       3.899688      -3.

In [435]:
# make sure the dataframe is g2g

reindexed_selections = make_dataframe_from_selections(selections)
reindexed_selections.head()

,date,perturbation,noon_air_temp,noon_ave_wind,noon_solar_rad,noon_air_temp_m1,noon_ave_wind_m1,noon_solar_rad_m1,noon_air_temp_m2,noon_ave_wind_m2,noon_solar_rad_m2
0,2022-07-03,0,1.807250,-1.529991,0.454718,2.744055,-2.396813,1.058133,1.204289,-1.949879,0.350326
1,2022-07-03,1,1.192633,-1.249993,0.048738,2.796073,-2.430842,1.121635,1.904569,-1.533377,0.789598
2,2022-07-03,2,2.697526,-0.612498,0.963957,2.010359,-3.892816,0.689188,1.084930,-2.010843,0.590375
3,2022-07-03,3,1.925265,-3.510717,0.004560,2.055168,-1.622532,1.121635,1.126183,-3.522782,0.147336
4,2022-07-03,4,0.104025,-1.528424,-0.700262,2.498967,-4.111766,0.932853,1.343561,-2.461409,0.719143


Everything from here down is hard to test, as it's depenedent on other fucntions.

### Prediction Functions

This model is an ensemble of mulitple leave-one-out (LOO) cross-validation (CV) models. We need the data from all the models to make a forecast.

In [444]:
# use the LOO CV to make predictions - the output here is a dictionary with 8 keys (pred_1, pred_2, ..., pred_8) and 31 x 8 (248) arrays
def make_day1_predictions(features, models):
    predictions = {}
    for i, model in enumerate(models, start=1):
        predictions[f'pred_{i}'] = model.predict(features)
    return predictions

# use the LOO CV to make predictions, this time we have to make sure we send the correct model features to the correct model for prediction
def make_all_other_predictions(features, models, date):
    predictions = pd.DataFrame()
    for i, model in enumerate(models, start=1):
        model_features = features[features.index.get_level_values(0) == i].copy()
        if i == 1:
            print(model_features.shape)
        pred = model.predict(model_features)
        temp_df = pd.DataFrame(pred, columns=['mean_1m_temp_degC', 'mean_0_5m_temp_degC'])
        temp_df['perturbation'] = temp_df.reset_index().index
        temp_df['model'] = i
        temp_df['date'] = date
        predictions = pd.concat([predictions, temp_df])
    return predictions

# create a dataframe from the predictions, adding the purturbation, model, date, and extracted temperature data
# predictions will be a dictionary with 8 prediciton keys and 31 x 8 (248) arrays
# obs_fore_less is a dataframe with 31 rows x 52 cols (met forecast makes 1 -> 31 for 31 members) (this is observations + forecasted met)
def create_day1_out_forecast_dataframe(predictions, obs_fore_less, date_valid):
    forecasted_temp = pd.DataFrame(columns=['date', 'perturbation', 'model', 'mean_1m_temp_degC', 'mean_0_5m_temp_degC'])
    for idx, (i, pred) in enumerate(predictions.items()):
        temp_df = pd.DataFrame({
            'perturbation': idx,
            'model': int(i.split('_')[1]),
            'mean_1m_temp_degC': [p[0] for p in pred],
            'mean_0_5m_temp_degC': [p[1] for p in pred],
            'date': date_valid
        })
        forecasted_temp = pd.concat([forecasted_temp, temp_df])
    return forecasted_temp

### Forecast Function

In [437]:
def make_seven_day_forecast(date):
    
    print(date)
    date = pd.to_datetime(date)
    
    # prepare the met forecast data for assimilation by shifting dates and calculating the offset
    fore = prepare_initial_forecast_data(date)   

    # make an emptydataframe to store the forecasted data
    forecasted_temp = pd.DataFrame(columns=['date', 'perturbation', 'model', 'mean_1m_temp_degC', 'mean_0_5m_temp_degC'])

    # we'll run a forecast for each day, since the following day's forecast will be based on the previous day's forecast, where `d` is the day offset from today
    for d in range(0, 7):
        
        # Setup for the iteration
        print("Forecasting day: ", d+1)
        
        # set the forecast date as the date today plus the offset (d)
        forecast_date = pd.to_datetime(date) + pd.DateOffset(days=d)
        # filter the test dataset (2022 data in this case) for the forecast date
        # we will use the inflow/interflow/pumping data from this dataframe
        obs = test[test["date"] == forecast_date].copy()
        
        # the first day will be a bit different from subsequent days - so we use a separate function for this
        if d == 0:
            
            obs_fore = process_day_zero(obs, fore)
            
            # preprocess the data into labels and features for the forecast
            features = obs_fore.drop(columns = ["date", "mean_1m_temp_degC", "mean_0_5m_temp_degC"])
            
            # make the forecast for each perturbation
            predictions = make_predictions(features, [model_1, model_2, model_3, model_4, model_5, model_6, model_7, model_8])
            
            # and now we need to create the dataframe for the next iteration
            next_day_forecast = create_forecast_dataframe(predictions, obs_fore, forecast_date)
        
        else:
            # remove the noon met data and the observed temperature data from the days that need to be filled with forecasted data for days 2-7 of the forecast
            obs_less = drop_columns(obs, d)
            
            # grab the forecasted met data for the offset date the merge with the observed data
            selected_met = met_data_selection(fore, d)

            reindexed_met = make_dataframe_from_selections(selected_met)

            # make the data forecast ready by incorporating the forecasted met data and forecasted temperature data 
            forecast_ready = make_forecast_ready(obs_less, reindexed_met, forecasted_temp, forecast_date, d)

            # make the forecast for each perturbation
            forecast = make_predictions(forecast_ready, [model_1, model_2, model_3, model_4, model_5, model_6, model_7, model_8])

            # and create the dataframe for the next iteration
            next_day_forecast = create_forecast_dataframe(forecast, obs_less, forecast_date)

        # Append to the main dataframe
        forecasted_temp = pd.concat([forecasted_temp, next_day_forecast])
        
    return forecasted_temp


In [438]:
# here, we'll walk through the innards of the forecast function to make sure everything is working as expected

print(date)
date = pd.to_datetime(date)

# prepare the met forecast data for assimilation by shifting dates and calculating the offset
fore = prepare_initial_forecast_data(date)   

# make an emptydataframe to store the forecasted data
forecasted_temp = pd.DataFrame(columns=['date', 'perturbation', 'model', 'mean_1m_temp_degC', 'mean_0_5m_temp_degC'])

# print off the forecast met data
fore.head()


2022-07-01


,date,number,noon_air_temp,noon_ave_wind,noon_solar_rad,offset
13237,2022-07-01,0,1.204289,-1.949879,0.350326,0
13238,2022-07-02,0,2.744055,-2.396813,1.058133,1
13239,2022-07-03,0,1.807250,-1.529991,0.454718,2
13240,2022-07-04,0,2.064673,-1.473007,0.561042,3
13241,2022-07-05,0,4.694532,-8.678423,1.068677,4


In [447]:
# to test the for-loop of forecast days, we'll just run d = 0 to test the initial forecast
d = 0

# Setup for the iteration
print("Forecasting day: ", d+1)

# set the forecast date as the date today plus the offset (d)
forecast_date = pd.to_datetime(date) + pd.DateOffset(days=d)
# filter the test dataset (2022 data in this case) for the forecast date
# we will use the inflow/interflow/pumping data from this dataframe
obs = test[test["date"] == forecast_date].copy()

# the first day will be a bit different from subsequent days - so we use a separate function for this
if d == 0:
    # process the data for the first day
    obs_fore = process_day_zero(obs, fore)
    
    # preprocess the data into labels and features for the forecast
    features = obs_fore.drop(columns = ["date", "mean_1m_temp_degC", "mean_0_5m_temp_degC"])
    
    # make the forecast for each perturbation
    forecast = make_day1_predictions(features, [model_1, model_2, model_3, model_4, model_5, model_6, model_7, model_8])
    
    # and now we need to create the dataframe for the next iteration
    next_day_forecast = create_day1_out_forecast_dataframe(forecast, obs_fore, date)

    # Append to the main dataframe
    forecasted_temp = pd.concat([forecasted_temp, next_day_forecast])
    
else:
    # remove the noon met data and the observed temperature data from the days that need to be filled with forecasted data for days 2-7 of the forecast
    obs_less = drop_columns(obs, d)
    
    # grab the forecasted met data for the offset date the merge with the observed data
    selected_met = met_data_selection(fore, d)

    # collate the selected met data into a dataframe
    reindexed_met = make_dataframe_from_selections(selected_met)
    
    # make the data forecast ready by incorporating the forecasted met data and forecasted temperature data 
    forecast_ready = make_forecast_ready(obs_less, reindexed_met, forecasted_temp, forecast_date, d)
    
    print(forecast_ready.shape)

    # make the forecast for each perturbation
    forecast = make_all_other_predictions(forecast_ready, [model_1, model_2, model_3, model_4, model_5, model_6, model_7, model_8], forecast_date)
    
    print(forecast.shape, forecasted_temp.shape)
    # Append to the main dataframe
    forecasted_temp = pd.concat([forecasted_temp, forecast])

next_day_forecast.head(), next_day_forecast.shape

Forecasting day:  1


/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_29928/4123300601.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  forecasted_temp = pd.concat([forecasted_temp, temp_df])


(        date perturbation model  mean_1m_temp_degC  mean_0_5m_temp_degC
 0 2022-07-01            0     1           0.448756             0.381408
 1 2022-07-01            0     1           0.531312             0.458859
 2 2022-07-01            0     1           0.446059             0.378736
 3 2022-07-01            0     1           0.485061             0.290643
 4 2022-07-01            0     1           0.507616             0.376304,
 (248, 5))

In [453]:
forecasted_temp = pd.DataFrame(columns=['date', 'perturbation', 'model', 'mean_1m_temp_degC', 'mean_0_5m_temp_degC'])

# okay, and let's run the whole loop
# we'll run a forecast for each day, since the following day's forecast will be based on the previous day's forecast, where `d` is the day offset from today
for d in range(0,4):
    
    # Setup for the iteration
    print("Forecasting day: ", d+1)
    
    # set the forecast date as the date today plus the offset (d)
    forecast_date = pd.to_datetime(date) + pd.DateOffset(days=d)
    # filter the test dataset (2022 data in this case) for the forecast date
    # we will use the inflow/interflow/pumping data from this dataframe
    obs = test[test["date"] == forecast_date].copy()
    
    # the first day will be a bit different from subsequent days - so we use a separate function for this
    if d == 0:
        # process the data for the first day
        obs_fore = process_day_zero(obs, fore)
        
        # preprocess the data into labels and features for the forecast
        features = obs_fore.drop(columns = ["date", "mean_1m_temp_degC", "mean_0_5m_temp_degC"])
        
        # make the forecast for each perturbation
        forecast = make_day1_predictions(features, [model_1, model_2, model_3, model_4, model_5, model_6, model_7, model_8])
        
        # and now we need to create the dataframe for the next iteration
        next_day_forecast = create_day1_out_forecast_dataframe(forecast, obs_fore, date)

        # Append to the main dataframe
        forecasted_temp = pd.concat([forecasted_temp, next_day_forecast])
        
    else:
        # remove the noon met data and the observed temperature data from the days that need to be filled with forecasted data for days 2-7 of the forecast
        obs_less = drop_columns(obs, d)
        
        # grab the forecasted met data for the offset date the merge with the observed data
        selected_met = met_data_selection(fore, d)

        # collate the selected met data into a dataframe
        reindexed_met = make_dataframe_from_selections(selected_met)
        
        # make the data forecast ready by incorporating the forecasted met data and forecasted temperature data 
        forecast_ready = make_forecast_ready(obs_less, reindexed_met, forecasted_temp, forecast_date, d)
        
        # make the forecast for each perturbation
        forecast = make_all_other_predictions(forecast_ready, [model_1, model_2, model_3, model_4, model_5, model_6, model_7, model_8], forecast_date)
        
        # Append to the main dataframe
        forecasted_temp = pd.concat([forecasted_temp, forecast])
    
    print("forecasted_temp shape:")
    print(forecasted_temp.shape)

Forecasting day:  1
forecasted_temp shape:
(248, 5)
Forecasting day:  2
          date  perturbation model  mean_1m_temp_degC_m1  \
0   2022-07-02             0     1              0.448756   
1   2022-07-02             0     1              0.531312   
2   2022-07-02             0     1              0.446059   
3   2022-07-02             0     1              0.485061   
4   2022-07-02             0     1              0.507616   
..         ...           ...   ...                   ...   
243 2022-07-02             7     8              0.491750   
244 2022-07-02             7     8              0.394008   
245 2022-07-02             7     8              0.328762   
246 2022-07-02             7     8              0.326857   
247 2022-07-02             7     8              0.494641   

     mean_0_5m_temp_degC_m1  mean_1m_temp_degC_m2  mean_0_5m_temp_degC_m2  \
0                  0.381408              0.116824                0.439962   
1                  0.458859              0.116824    

/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_29928/4123300601.py:36: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  forecasted_temp = pd.concat([forecasted_temp, temp_df])
/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_29928/1652822583.py:31: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  forecasted_temp = pd.concat([forecasted_temp, next_day_forecast])


forecasted_temp shape:
(496, 5)
Forecasting day:  3
          date  perturbation model  mean_1m_temp_degC_m1  \
0   2022-07-03             0     1              0.977364   
1   2022-07-03             1     1              1.003484   
2   2022-07-03             2     1              0.976499   
3   2022-07-03             3     1              0.978239   
4   2022-07-03             4     1              0.990858   
..         ...           ...   ...                   ...   
243 2022-07-03            26     8              0.928045   
244 2022-07-03            27     8              0.903053   
245 2022-07-03            28     8              0.891839   
246 2022-07-03            29     8              0.887109   
247 2022-07-03            30     8              0.936801   

     mean_0_5m_temp_degC_m1  mean_1m_temp_degC_m3  mean_0_5m_temp_degC_m3  \
0                  0.628538              0.113509                0.437101   
1                  0.660029              0.113509                0.437101

In [ ]:
   # Append to the main dataframe
    forecasted_temp = pd.concat([forecasted_temp, next_day_forecast])

# Make a few forecasts

In [135]:
jul07_seven_day = make_seven_day_forecast("2022-07-07")
aug20_seven_day = make_seven_day_forecast("2022-08-20")
sept11_seven_day = make_seven_day_forecast("2022-09-11")

2022-07-07
Forecasting day:  1
Forecasting day:  2
         date  mean_1m_temp_degC_m2  mean_0_5m_temp_degC_m2  \
40 2022-07-08              1.017169                1.231476   

    mean_1m_temp_degC_m3  mean_0_5m_temp_degC_m3  pump_q_m1  pump_q_m2  \
40               1.01337                1.218688  -1.049545  -1.041707   

    sum_pump_q_p2  max_pump_q_p2  sum_pump_q_p7  ...  noon_solar_rad_m5  \
40      -1.127968      -1.164634      -1.188506  ...           1.207901   

    noon_solar_rad_m6  noon_solar_rad_m7  min_chip_q_m1  min_chip_q_m2  \
40           1.429743           1.129624       0.891415       0.662143   

    min_chip_q_p7   NF_q_m1   NF_q_m2  mean_NF_q_p7  max_NF_q_p7  
40        0.36899  0.924966  1.126431       0.47867     0.758203  

[1 rows x 42 columns]         date  perturbation  noon_air_temp_m1  noon_ave_wind_m1  \
0 2022-07-08             0          2.496350         -4.044584   
1 2022-07-08             1          2.348164         -2.627897   
2 2022-07-08      

/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_29928/1956332727.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  forecasted_temp = pd.concat([forecasted_temp, temp_df])
/var/folders/x8/v7bmckc139v2dqcm04jgfjlr0000gn/T/ipykernel_29928/2077405827.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  forecasted_temp = pd.concat([forecasted_temp, next_day_forecast])


KeyError: "['noon_air_temp', 'noon_ave_wind', 'noon_solar_rad'] not in index"

# Calculate actual values from the forecasted data

I'm not sure why this creates dupes, but it does, i'll have to figure that out later.

In [ ]:
jul07_seven_day

In [ ]:
mean_std = pd.read_csv(os.path.join(data_dir, "mean_std_train_val_t2022_v2024-10-28.csv"))
mean_std = mean_std.set_index("Unnamed: 0")

jul07_seven_day["mean_1m_temp_degC"] = calculate_vals(jul07_seven_day["mean_1m_temp_degC"], mean_std.loc["mean_1m_temp_degC", "mean"], mean_std.loc["mean_1m_temp_degC", "std"])
jul07_seven_day["mean_0_5m_temp_degC"] = calculate_vals(jul07_seven_day["mean_0_5m_temp_degC"], mean_std.loc["mean_0_5m_temp_degC", "mean"], mean_std.loc["mean_0_5m_temp_degC", "std"])

aug20_seven_day["mean_1m_temp_degC"] = calculate_vals(aug20_seven_day["mean_1m_temp_degC"], mean_std.loc["mean_1m_temp_degC", "mean"], mean_std.loc["mean_1m_temp_degC", "std"])
aug20_seven_day["mean_0_5m_temp_degC"] = calculate_vals(aug20_seven_day["mean_0_5m_temp_degC"], mean_std.loc["mean_0_5m_temp_degC", "mean"], mean_std.loc["mean_0_5m_temp_degC", "std"])

sept11_seven_day["mean_1m_temp_degC"] = calculate_vals(sept11_seven_day["mean_1m_temp_degC"], mean_std.loc["mean_1m_temp_degC", "mean"], mean_std.loc["mean_1m_temp_degC", "std"])
sept11_seven_day["mean_0_5m_temp_degC"] = calculate_vals(sept11_seven_day["mean_0_5m_temp_degC"], mean_std.loc["mean_0_5m_temp_degC", "mean"], mean_std.loc["mean_0_5m_temp_degC", "std"])


In [ ]:
# save a csv of those forecasts
jul07_seven_day.to_csv("~/Documents/GitHub/ats-data-driven-forecasting/run-operational/output/jul07_seven_day.csv", index=False)
aug20_seven_day.to_csv("~/Documents/GitHub/ats-data-driven-forecasting/run-operational/output/aug20_seven_day.csv", index=False)
sept11_seven_day.to_csv("~/Documents/GitHub/ats-data-driven-forecasting/run-operational/output/sept11_seven_day.csv", index=False)